In [28]:
from tqdm import tqdm_notebook as tqdm

In [11]:
import us, feather

In [29]:
import requests, json

In [66]:
dfs = []
years = [2010, 2011, 2012, 2014, 2016]

In [67]:
for year in tqdm(years):
    try:
        url = "https://api.census.gov/data/{}/acs/acs5/profile?get=DP02_0067PE,NAME&for=congressional%20district:*".format(year)
        df = pd.read_json((requests.get(url).content))
        df.columns = df.iloc[0].values
        df = df.iloc[1:].copy()
        df['year'] = year
        dfs.append(df)
    except:
        pass

In [68]:
df = pd.concat(dfs)

In [69]:
df.columns = ['percent_bachelor_or_above', 'name', 'state_fips', 'district', 'year']

In [113]:
df1 = pd.read_csv('ACS_06_EST_S1501/ACS_06_EST_S1501_with_ann.csv')

col = ['HC01_EST_VC05', 'GEO.id2', 'GEO.display-label']

df1 = df1[col].iloc[1:].copy()
df1['year'] = 2006

df2 = pd.read_csv('ACS_08_3YR_S1501/ACS_08_3YR_S1501_with_ann.csv')

df2 = df2[col].iloc[1:].copy()
df2['year'] = 2008

df3 = pd.read_csv('DEC_00_SF3_DP2/DEC_00_SF3_DP2_with_ann.csv')

col = ['HC01_VC18', 'GEO.id2', 'GEO.display-label']
df3 = df3[col].iloc[1:].copy()
df3['year'] = 2000

col += ['year']
df1.columns = col
df2.columns = col
df3.columns = col

In [114]:
df_other = pd.concat([df1, df2, df3])

In [115]:
df_other['state_fips'] = df_other['GEO.id2'].apply(lambda x : x[:2])
df_other['district'] = df_other['GEO.id2'].apply(lambda x : x[2:])

In [116]:
df_other = df_other.drop('GEO.id2', axis=1)

In [117]:
df_other.columns = ['percent_bachelor_or_above', 'name', 'year', 'state_fips', 'district']

In [119]:
df = pd.concat([df, df_other], sort=False)

In [135]:
df['name'].str.findall(r'\(([^\)]+?th.+?)\)').apply(lambda x : x[0]).value_counts()

110th Congress    437
115th Congress    437
111th Congress    437
113th Congress    437
112th Congress    437
109th Congress    437
114th Congress    437
106th Congress    436
Name: name, dtype: int64

In [86]:
df.reset_index().to_feather('percent_bachelor_or_above.feather')

In [29]:
df = feather.read_dataframe('percent_bachelor_or_above.feather').iloc[:,1:]

In [30]:
df['congress_session'] = df['name'].str.findall(r'\(([^\)]+?th.+?)\)').apply(lambda x : x[0])

In [31]:
df.to_csv('education.csv', index=False)

In [32]:
elections = feather.read_dataframe('past_house.feather')

In [33]:
elections = elections[['state', 'area', 'raceyear', 'repcandidate', 'demcandidate']].copy()

In [34]:
df[df['name'].str.contains('Alaska')]

,percent_bachelor_or_above,name,state_fips,district,year,congress_session
101,27.0,Congressional District (at Large) (111th Congr...,02,00,2010.0000,111th Congress
444,27.2,Congressional District (at Large) (112th Congr...,02,00,2011.0000,112th Congress
1061,27.5,Congressional District (at Large) (113th Congr...,02,00,2012.0000,113th Congress
1694,27.7,Congressional District (at Large) (114th Congr...,02,00,2014.0000,114th Congress
1755,28.8,Congressional District (at Large) (115th Congr...,02,00,2016.0000,115th Congress
2192,4.5,Congressional District (at Large) (109th Congr...,02,00,2006.0000,109th Congress
2629,5.0,Congressional District (at Large) (110th Congr...,02,00,2008.0000,110th Congress
3066,24.7,Congressional District (at Large) (106th Congr...,02,00,2000.0000,106th Congress


In [35]:
elections['state_fips'] = elections['state'].apply(lambda x: us.states.lookup(x).fips)

In [36]:
def clean_district(x):
    lst = x.split(' ')
    t = lst[1:]
    if len(t) == 1: 
        try:
            return str(int(t[0])).zfill(2)
        except:
            return '00'

In [37]:
elections['district'] = elections['area'].apply(clean_district)

In [38]:
df['year'] = df['year'].astype(int)

In [39]:
df['state_fips'] = df['state_fips'].astype(str)
df['district'] = df['district'].astype(str)

In [40]:
elections['state_fips'] = elections['state_fips'].astype(str)
elections['district'] = elections['district'].astype(str)

In [45]:
years = {'106th Congress': 1998, 
 '109th Congress': 2004, 
 '110th Congress': 2006, 
 '111th Congress': 2008,
 '112th Congress': 2010, 
 '113th Congress': 2012, 
 '114th Congress': 2014, 
 '115th Congress': 2016}

In [46]:
df['raceyear'] = df['congress_session'].apply(lambda x : years[x])

In [50]:
df.merge(elections, on=['state_fips', 'district', 'raceyear'], how='left')

,percent_bachelor_or_above,name,state_fips,district,year,congress_session,raceyear,state,area,repcandidate,demcandidate
0,17.3,"Congressional District 1 (111th Congress), Nevada",32,01,2010,111th Congress,2008,Nevada,District 1,"Wegner, Kenneth","Berkley, Shelley"
1,22.2,"Congressional District 2 (111th Congress), Nevada",32,02,2010,111th Congress,2008,Nevada,District 2,"Heller, Dean","Derby, Jill"
2,25.0,"Congressional District 3 (111th Congress), Nevada",32,03,2010,111th Congress,2008,Nevada,District 3,"Porter, Jon","Titus, Dina"
3,32.9,"Congressional District 1 (111th Congress), New...",33,01,2010,111th Congress,2008,New Hampshire,District 1,"Bradley, Jeb","Shea-Porter, Carol"
4,32.8,"Congressional District 2 (111th Congress), New...",33,02,2010,111th Congress,2008,New Hampshire,District 2,"Horn, Jennifer","Hodes, Paul W."
5,25.0,"Congressional District 1 (111th Congress), New...",34,01,2010,111th Congress,2008,New Jersey,District 1,"Glading, Dale M.","Andrews, Robert E."
6,21.9,"Congressional District 2 (111th Congress), New...",34,02,2010,111th Congress,2008,New Jersey,District 2,"LoBiondo, Frank A.","Kurkowski, David"
7,31.9,"Congressional District 3 (111th Congress), New...",34,03,2010,111th Congress,2008,New Jersey,District 3,"Myers, Chris","Adler, John"
8,29.6,"Congressional District 4 (111th Congress), New...",34,04,2010,111th Congress,2008,New Jersey,District 4,"Smith, Christopher H.","Zeitz, Joshua M."
9,43.3,"Congressional District 5 (111th Congress), New...",34,05,2010,111th Congress,2008,New Jersey,District 5,"Garrett, Scott","Shulman, Dennis"


In [49]:
df

,percent_bachelor_or_above,name,state_fips,district,year,congress_session,raceyear
0,17.3,"Congressional District 1 (111th Congress), Nevada",32,01,2010,111th Congress,2008
1,22.2,"Congressional District 2 (111th Congress), Nevada",32,02,2010,111th Congress,2008
2,25.0,"Congressional District 3 (111th Congress), Nevada",32,03,2010,111th Congress,2008
3,32.9,"Congressional District 1 (111th Congress), New...",33,01,2010,111th Congress,2008
4,32.8,"Congressional District 2 (111th Congress), New...",33,02,2010,111th Congress,2008
5,25.0,"Congressional District 1 (111th Congress), New...",34,01,2010,111th Congress,2008
6,21.9,"Congressional District 2 (111th Congress), New...",34,02,2010,111th Congress,2008
7,31.9,"Congressional District 3 (111th Congress), New...",34,03,2010,111th Congress,2008
8,29.6,"Congressional District 4 (111th Congress), New...",34,04,2010,111th Congress,2008
9,43.3,"Congressional District 5 (111th Congress), New...",34,05,2010,111th Congress,2008
